In [1]:
# (C) Copyright Aaron Goldberg, 2022.
#
# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.

In [1]:
import strawberryfields as sf
from strawberryfields.ops import Sgate, Rgate, BSgate, MeasureFock
from strawberryfields.tdm import borealis_gbs, full_compile, get_mode_indices
import numpy as np

2023-01-03 16:36:57.883672: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Compilation will make sure the specs match an actual device, so must be done using a particular device and its characteristics
eng = sf.RemoteEngine("borealis")
device = eng.device

In [3]:
# Let's define our circuit manually
modes = 2

# squeezing-gate parameters. These will be rounded to the closest one supported by hardware, so that can by verified
#r0 = device.certificate["squeezing_parameters_mean"]["high"]
r = [1.2] * modes

# rotation-gate parameters
phi_0 = [0.] * modes
phi_1 = [0.] * modes
phi_2 = [0.] * modes
phi_0[1] = -np.pi/2 # Makes sure we have a 50:50 BS instead of a symmetric one

# beamsplitter parameters. Set irrelevant transmission parameters to 0 to bypass all nonessential loops and mitigate losses
T_0 = [0.] * modes
T_1 = [0.] * modes
T_2 = [0.] * modes
T_0[1] = 0.5 # 50:50 BS
alpha_0 = np.arccos(np.sqrt(T_0))
alpha_1 = np.arccos(np.sqrt(T_1))
alpha_2 = np.arccos(np.sqrt(T_2))

# the travel time per delay line in time bins
delay_0, delay_1, delay_2 = 1, 6, 36

# set the first beamsplitter arguments to 'T=1' ('alpha=0') to fill the
# loops with pulses
alpha_0[:delay_0] = 0.0
alpha_1[:delay_1] = 0.0
alpha_2[:delay_2] = 0.0

# The gate arguments need to be defined as lists, so if they were defined with numpy we need to cast them to lists
gate_args = {
    "Sgate": r,
    "loops": {
        0: {"Rgate": phi_0, "BSgate": alpha_0.tolist()},
        1: {"Rgate": phi_1, "BSgate": alpha_1.tolist()},
        2: {"Rgate": phi_2, "BSgate": alpha_2.tolist()},
    },
}

In [4]:
# Now compile:
gate_args_list = full_compile(gate_args, device)

In [5]:
# Verify components if desired
#gate_args_list[2]

In [6]:
delays = [1, 6, 36]
vac_modes = sum(delays)

n, N = get_mode_indices(delays)

In [7]:
prog = sf.TDMProgram(N)

with prog.context(*gate_args_list) as (p, q):
    Sgate(p[0]) | q[n[0]]
    #LossChannel(eta_glob) | q[n[0]]
    for i in range(len(delays)):
        Rgate(p[2 * i + 1]) | q[n[i]]
        BSgate(p[2 * i + 2], np.pi / 2) | (q[n[i + 1]], q[n[i]]) # Get rid of phases to make regular 50:50 BS instead of symmetric BS
        #LossChannel(etas_loop[i]) | q[n[i]]
    #LossChannel(p[7]) | q[0]
    MeasureFock() | q[0]

In [8]:
shots = 10**6
results = eng.run(prog, shots=shots, crop=True)

In [9]:
shots=len(results.samples) # This is in case we exceeded a maximum
print(shots)

966000


In [10]:
np.savetxt(f'QCS_SV_r{gate_args_list[0][0]}.csv', np.reshape(results.samples,[shots, modes]).astype(int), delimiter=',',fmt='%i')

In [50]:
# Count how many times each detector said a given number of photons; this is <n|rho|n> for each mode
max_photon=np.amax(results.samples)
print("Highest number recorded on a detector is: ",max_photon)
#all_my_probs=[np.count_nonzero(results.samples == n, axis=0)/shots for n in range(max_photon+1)]
# We only need to look at mode 1 for determining the QCS. This is the mode where the creation/annihilation operators get converted into the difference a-b, up to a global phase here or a relative phase between the final two modes, as can be verified by the transfer matrix
my_probs=[np.count_nonzero(results.samples[:,:,1] == n)/shots for n in range(max_photon+1)] 

Highest number recorded on a detector is:  12


In [48]:
print("Number of modes:",modes,"Average number of photons detected per mode: ",np.sum(results.samples)/shots/modes)
print("Quadrature coherence scale from mode 1:",1.+2*np.sum([n*((-1)**n) * my_probs[n] for n in range(max_photon+1)])/np.sum([((-1)**n) * my_probs[n] for n in range(max_photon+1)]))
print("If it was pure we'd have QCS of 1+2 nbar (with nbar from squeezed vacuum):",1.+2.*np.sinh(gate_args_list[0][0])**2)
print("nbar of mode 1",np.sum([n * my_probs[n] for n in range(max_photon+1)]))
print("purity of initial state from output Fock",np.sum([((-1)**n) * my_probs[n] for n in range(max_photon+1)]))
print("g^2(0) mode 1:",np.sum([n*n*my_probs[n] for n in range(max_photon+1)])/(np.sum([n * my_probs[n] for n in range(max_photon+1)])**2)-1/(np.sum([n * my_probs[n] for n in range(max_photon+1)])))

Number of modes: 2 Average number of photons detected per mode:  0.41680904522613066
Quadrature coherence scale from mode 1: 0.7579166743512661
If it was pure we'd have QCS of 1+2 nbar (with nbar from squeezed vacuum): 5.086842953286617
nbar of mode 1 0.3794532663316582
purity of initial state from output Fock 0.6539216080402009
g^2(0) mode 1: 3.2445779681137634


In [49]:
device.certificate

{'target': 'borealis',
 'finished_at': '2022-12-15T18:41:33.367752+00:00',
 'loop_phases': [-1.758, 0.873, 1.917],
 'schmidt_number': 1.145,
 'common_efficiency': 0.386,
 'loop_efficiencies': [0.887, 0.827, 0.727],
 'squeezing_parameters_mean': {'low': 0.658,
  'high': 1.155,
  'medium': 0.979,
  'zero': 0},
 'relative_channel_efficiencies': [0.925,
  0.933,
  0.908,
  0.998,
  0.968,
  0.916,
  0.897,
  0.969,
  0.956,
  0.96,
  0.96,
  1.0,
  0.938,
  0.968,
  0.962,
  0.907]}